<a href="https://colab.research.google.com/github/OswaldGutierrez/Modelos-IA/blob/main/RegresionLogistica_NATICUSdroid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size="6">PROYECTO IA 2025-2</font>


<font size="3">**1. IMPORTACIÓN DE LIBRERÍAS NECESARIAS**</font>


In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, confusion_matrix,
                             classification_report, make_scorer)
import warnings
warnings.filterwarnings('ignore')

<font size="3">**2. CARACTERÍSTICAS DEL CONJUNTO DE DATOS**</font>


In [2]:
!wget https://raw.githubusercontent.com/OswaldGutierrez/dataNATICUSdroid/main/naticusdroid+android+permissions+dataset.zip
!unzip naticusdroid+android+permissions+dataset.zip
d = pd.read_csv("data.csv")

--2025-11-24 13:15:23--  https://raw.githubusercontent.com/OswaldGutierrez/dataNATICUSdroid/main/naticusdroid+android+permissions+dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5136681 (4.9M) [application/zip]
Saving to: ‘naticusdroid+android+permissions+dataset.zip’

naticusdroid+androi 100%[===================>]   4.90M  --.-KB/s    in 0.1s    

2025-11-24 13:15:23 (45.2 MB/s) - ‘naticusdroid+android+permissions+dataset.zip’ saved [5136681/5136681]

Archive:  naticusdroid+android+permissions+dataset.zip
 extracting: data.csv                


In [3]:
data = pd.read_csv('data.csv')
print(f"Dataset de Permisos de Android cargado: {data.shape[0]} filas, {data.shape[1]} columnas")

Dataset de Permisos de Android cargado: 29332 filas, 87 columnas


In [4]:
# Separar características (X) y etiqueta (y)
X = data.drop('Result', axis=1)
y = data['Result']

print(f"  - Características: {X.shape[1]}")
print(f"  - Muestras: {len(y)}")

  - Características: 86
  - Muestras: 29332


<font size="3">**3. NORMALIZACIÓN DE CARACTERÍSTICAS**</font>


In [5]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Características normalizadas (media=0, std=1)")

Características normalizadas (media=0, std=1)


<font size="3">**4. DEFINICIÓN DE HIPERPARÁMETROS**</font>


In [6]:
# Configuración de hiperparámetros para Regresión Logística
hyperparameters = {
    'C': 1.0,                    # Inverso de regularización (menor = más regularización)
    'penalty': 'l2',             # Tipo de regularización (l1, l2)
    'solver': 'lbfgs',           # Algoritmo de optimización
    'max_iter': 1000,            # Número máximo de iteraciones
    'random_state': 42,          # Semilla para reproducibilidad
    'class_weight': None         # No es necesario ya que nuestro dataset ya está balanceado
}

print("  Hiperparámetros seleccionados:")
for key, value in hyperparameters.items():
    print(f"    - {key}: {value}")

# Creamos el modelo
model = LogisticRegression(**hyperparameters)

  Hiperparámetros seleccionados:
    - C: 1.0
    - penalty: l2
    - solver: lbfgs
    - max_iter: 1000
    - random_state: 42
    - class_weight: None


<font size="3">**5. MÉTRICAS DE DESEMPEÑO**</font>


In [7]:
# Métricas a evaluar
scoring_metrics = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

print("  Métricas seleccionadas:")
print("    - Accuracy:  Precisión general del modelo")
print("    - Precision: De lo predicho como malware, cuánto es realmente malware")
print("    - Recall:    De todo el malware real, cuánto detectamos (CRÍTICO)")
print("    - F1-Score:  Balance entre Precision y Recall")
print("    - ROC-AUC:   Capacidad de discriminación del modelo")

  Métricas seleccionadas:
    - Accuracy:  Precisión general del modelo
    - Precision: De lo predicho como malware, cuánto es realmente malware
    - Recall:    De todo el malware real, cuánto detectamos (CRÍTICO)
    - F1-Score:  Balance entre Precision y Recall
    - ROC-AUC:   Capacidad de discriminación del modelo


<font size="3">**6. MÉTODO 10-FOLD CROSS-VALIDATION**</font>


In [8]:
# ENTRENAR CON 10-FOLD CROSS-VALIDATION
# Configurar validación cruzada estratificada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Entrenar y evaluar con cross-validation
print("  Ejecutando 10-fold cross-validation...")
cv_results = cross_validate(
    model,
    X_scaled,
    y,
    cv=cv,
    scoring=scoring_metrics,
    return_train_score=True,
    n_jobs=-1
)

print("  ✓ Cross-validation completado")

  Ejecutando 10-fold cross-validation...
  ✓ Cross-validation completado


<font size="3">**7. RESULTADOS**</font>


In [9]:
# Calcular estadísticas de cada métrica
print("\nMétricas en conjunto de TEST (promedio de 10 folds):")
print("-" * 60)

for metric_name in scoring_metrics.keys():
    test_scores = cv_results[f'test_{metric_name}']
    mean = test_scores.mean()
    std = test_scores.std()

    # Formatear nombre de métrica
    display_name = metric_name.replace('_', '-').upper()

    print(f"{display_name:12s}: {mean:.4f} (+/- {std:.4f})")

print("-" * 60)

# Métricas de entrenamiento (para detectar overfitting)
print("\nMétricas en conjunto de TRAIN (promedio de 10 folds):")
print("-" * 60)

for metric_name in scoring_metrics.keys():
    train_scores = cv_results[f'train_{metric_name}']
    mean = train_scores.mean()
    std = train_scores.std()

    display_name = metric_name.replace('_', '-').upper()
    print(f"{display_name:12s}: {mean:.4f} (+/- {std:.4f})")

print("-" * 60)


Métricas en conjunto de TEST (promedio de 10 folds):
------------------------------------------------------------
ACCURACY    : 0.9601 (+/- 0.0050)
PRECISION   : 0.9555 (+/- 0.0060)
RECALL      : 0.9654 (+/- 0.0045)
F1          : 0.9604 (+/- 0.0049)
ROC-AUC     : 0.9888 (+/- 0.0018)
------------------------------------------------------------

Métricas en conjunto de TRAIN (promedio de 10 folds):
------------------------------------------------------------
ACCURACY    : 0.9609 (+/- 0.0006)
PRECISION   : 0.9565 (+/- 0.0007)
RECALL      : 0.9659 (+/- 0.0006)
F1          : 0.9612 (+/- 0.0006)
ROC-AUC     : 0.9895 (+/- 0.0002)
------------------------------------------------------------


<font size="3">**8. ENTRENAMIENTO**</font>


In [10]:
# Entrenar modelo con todos los datos para predicciones futuras
final_model = LogisticRegression(**hyperparameters)
final_model.fit(X_scaled, y)

print("Modelo final entrenado")

Modelo final entrenado


<font size="3">**9. ANÁLISIS**</font>


In [11]:
# Importancia de características (top 10)
print("\nTop 10 características más importantes (Por valor absoluto de coeficientes):")
print("-" * 60)

feature_importance = np.abs(final_model.coef_[0])
feature_names = X.columns
top_indices = np.argsort(feature_importance)[-10:][::-1]

for i, idx in enumerate(top_indices, 1):
    print(f"{i:2d}. {feature_names[idx]:50s} | {feature_importance[idx]:.4f}")


Top 10 características más importantes (Por valor absoluto de coeficientes):
------------------------------------------------------------
 1. android.permission.READ_PHONE_STATE                | 2.1704
 2. com.google.android.c2dm.permission.RECEIVE         | 1.9426
 3. com.android.launcher.permission.INSTALL_SHORTCUT   | 1.3208
 4. com.google.android.finsky.permission.BIND_GET_INSTALL_REFERRER_SERVICE | 1.1237
 5. android.permission.REQUEST_INSTALL_PACKAGES        | 0.9838
 6. android.permission.USE_FINGERPRINT                 | 0.8384
 7. android.permission.SEND_SMS                        | 0.7975
 8. com.android.vending.BILLING                        | 0.7381
 9. android.permission.ACCESS_FINE_LOCATION            | 0.7370
10. com.majeur.launcher.permission.UPDATE_BADGE        | 0.6570


<font size="3">**10. INTERPRETACIÓN DE LOS RESULTADOS**</font>


In [12]:
test_accuracy = cv_results['test_accuracy'].mean()
test_precision = cv_results['test_precision'].mean()
test_recall = cv_results['test_recall'].mean()
test_f1 = cv_results['test_f1'].mean()
test_auc = cv_results['test_roc_auc'].mean()

print("\nRESUMEN:")
print(f"  - El modelo tiene una exactitud del {test_accuracy*100:.2f}%")
print(f"  - De las apps predichas como malware, {test_precision*100:.2f}% son correctas (Precision)")
print(f"  - Detecta el {test_recall*100:.2f}% del malware real (Recall)")
print(f"  - F1-Score de {test_f1:.4f} indica {'buen' if test_f1 > 0.8 else 'moderado'} balance")
print(f"  - ROC-AUC de {test_auc:.4f} indica {'excelente' if test_auc > 0.9 else 'buena'} capacidad de discriminación")

# Análisis de overfitting
train_accuracy = cv_results['train_accuracy'].mean()
diff = train_accuracy - test_accuracy

print(f"\nANÁLISIS DE OVERFITTING:")
print(f"  - Accuracy en train: {train_accuracy*100:.2f}%")
print(f"  - Accuracy en test:  {test_accuracy*100:.2f}%")
print(f"  - Diferencia: {diff*100:.2f}%")

if diff < 0.05:
    print("Buen ajuste - No hay overfitting significativo")
elif diff < 0.10:
    print("Overfitting leve - Considerar más regularización")
else:
    print("Overfitting detectado - Aumentar regularización (reducir C)")


RESUMEN:
  - El modelo tiene una exactitud del 96.01%
  - De las apps predichas como malware, 95.55% son correctas (Precision)
  - Detecta el 96.54% del malware real (Recall)
  - F1-Score de 0.9604 indica buen balance
  - ROC-AUC de 0.9888 indica excelente capacidad de discriminación

ANÁLISIS DE OVERFITTING:
  - Accuracy en train: 96.09%
  - Accuracy en test:  96.01%
  - Diferencia: 0.08%
Buen ajuste - No hay overfitting significativo
